# Sintax directed translate

Design translation scheme (offset parameters Pascal style. That is, the parameters are put on the stack form left ro right)

### Grammar:

```
Def -> ID ID Resto
Resto -> ‘,’ Tipo ID Resto
Resto -> epsilon
Tipo -> INT
Tipo -> CHAR
Tipo -> FLOAT
```

### Example:

**Input:**

`f(int a, float b, char c)`

**Output:**

```
Offset de c = 4
Offset de b = 4 + sizeof(char)
Offset de a = 4 + sizeof(char) + sizeof(float)

```

# Implementing the lexical analyzer (Lexer)

First we have to import the lexer from sly:

In [400]:
from sly import Lexer

Next we have to define our lexical analyzer, creating a class that inherits from Lexer and define the tokens and literals.
To avoid the errors from blank characters, add it to ignore variable (in this case we will ignore blank spaces and tabs).
ID is defined using a *re* so be careful with that because that definition matches the types. So you have to add
the special as special cases of *ID* definitions.

In [401]:
class Scanner(Lexer):
    tokens = {ID, INT, FLOAT, CHAR}
    literals = {','}

    #ignore whitespace and tabulation
    ignore = ' \t'

    # Regular expression rules for tokens

    ID = r'[a-zA-Z][\w_]*'

    # special cases

    ID['int'] = INT
    ID['float'] = FLOAT
    ID['char'] = CHAR

    # Error handling rule

    def error(self, t):
        print("Illegal character {}".format(t.value[0]))
        self.index += 1

Test our Lexer analyzer:

In [402]:
if __name__ == '__main__':
    data = "int a, float b, char c"
    lexer = Scanner()
    for token in lexer.tokenize(data):
        print('type = {}, value = {}'.format(token.type, token.value))


type = INT, value = int
type = ID, value = a
type = ,, value = ,
type = FLOAT, value = float
type = ID, value = b
type = ,, value = ,
type = CHAR, value = char
type = ID, value = c


# Implementing syntactic analyzer (Parser)

To implement the grammar rules, we will implement a Parser first import Parser from sly

In [403]:
from sly import Parser

Now we will do the same that we do to implement the Lexer, create a new class that inherits from Parser, pass the tokens
from the lexer to our parser, and define the grammar rules.

In [404]:
class FunctionParser(Parser):
    tokens = Scanner.tokens

    @_('tipo ID resto')
    def def_(self, p):
        print('Offset de {} = 4'.format(p.ID) + p.resto)

    @_('"," tipo ID resto')
    def resto(self, p):
        print('Offset de {} = 4'.format(p.ID) + p.resto)
        return p.resto +' + sizeof({})'.format(p.tipo)
    @_('')
    def resto(self, p):
        return ''
    @_('INT', 'FLOAT', 'CHAR')
    def tipo(self, p):
        return p[0]

Testing the parser

In [405]:
if __name__ == '__main__':
    lexer = Scanner()
    parser = FunctionParser()
    string = "int a, float b, char c"
    parser.parse(lexer.tokenize(string))

Offset de c = 4
Offset de b = 4 + sizeof(char)
Offset de a = 4 + sizeof(char) + sizeof(float)
